In [ ]:
import numpy as np
import random
from numpy.random import randn
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_data = {
  'good': True,
  'bad': False,
  'happy': True,
  'sad': False,
  'not good': False,
  'not bad': True,
  'not happy': False,
  'not sad': True,
  'very good': True,
  'very bad': False,
  'very happy': True,
  'very sad': False,
  'i am happy': True,
  'this is good': True,
  'i am bad': False,
  'this is bad': False,
  'i am sad': False,
  'this is sad': False,
  'i am not happy': False,
  'this is not good': False,
  'i am not bad': True,
  'this is not sad': True,
  'i am very happy': True,
  'this is very good': True,
  'i am very bad': False,
  'this is very sad': False,
  'this is very happy': True,
  'i am good not bad': True,
  'this is good not bad': True,
  'i am bad not good': False,
  'i am good and happy': True,
  'this is not good and not happy': False,
  'i am not at all good': False,
  'i am not at all bad': True,
  'i am not at all happy': False,
  'this is not at all sad': True,
  'this is not at all happy': False,
  'i am good right now': True,
  'i am bad right now': False,
  'this is bad right now': False,
  'i am sad right now': False,
  'i was good earlier': True,
  'i was happy earlier': True,
  'i was bad earlier': False,
  'i was sad earlier': False,
  'i am very bad right now': False,
  'this is very good right now': True,
  'this is very sad right now': False,
  'this was bad earlier': False,
  'this was very good earlier': True,
  'this was very bad earlier': False,
  'this was very happy earlier': True,
  'this was very sad earlier': False,
  'i was good and not bad earlier': True,
  'i was not good and not happy earlier': False,
  'i am not at all bad or sad right now': True,
  'i am not at all good or happy right now': False,
  'this was not happy and not good earlier': False,
}

test_data = {
  'this is happy': True,
  'i am good': True,
  'this is not happy': False,
  'i am not good': False,
  'this is not bad': True,
  'i am not sad': True,
  'i am very good': True,
  'this is very bad': False,
  'i am very sad': False,
  'this is bad not good': False,
  'this is good and happy': True,
  'i am not good and not happy': False,
  'i am not at all sad': True,
  'this is not at all good': False,
  'this is not at all bad': True,
  'this is good right now': True,
  'this is sad right now': False,
  'this is very bad right now': False,
  'this was good earlier': True,
  'i was not happy and not good earlier': False,
}

In [ ]:
# Create the vocabulary
vocab = list(set([w for text in train_data.keys() for w in text.split()]))
vocab_size = len(vocab)
print("%d unique words found" %vocab_size)

18 unique words found


In [ ]:
# Assign indices to each word
word_to_index = {w: i for i, w in enumerate(vocab)}
index_to_word = {i: w for i, w in enumerate(vocab)}

print(word_to_index["good"])
print(index_to_word[0])

10
is


In [ ]:
# Transform words into vectors
def createInputs(text):
  inputs = []
  for word in text.split():
    v = np.zeros((vocab_size, 1))
    v[word_to_index[word]] = 1
    inputs.append(v)

  return inputs

In [ ]:
# Defining RNN architecture
class RNN:
  def __init__(self, input_size, output_size, hidden_size=64):
    # Weights
    self.Whh = randn(hidden_size, hidden_size) / 1000
    self.Wxh = randn(hidden_size, input_size) / 1000
    self.Why = randn(output_size, hidden_size) / 1000

    # Biases
    self.bh = np.zeros((hidden_size, 1))
    self.by = np.zeros((output_size, 1))

  # Forward pass
  def forward(self, inputs):
    h = np.zeros((self.Whh.shape[0], 1))

    self.last_inputs = inputs
    self.last_hs = { 0: h }

    # Perform each step of the RNN
    for i, x in enumerate(inputs):
      h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
      self.last_hs[i + 1] = h

    # Compute the output
    y = self.Why @ h + self.by

    return y, h

  # Back propagation
  def backprop(self, d_y, learn_rate=2e-2):
    n = len(self.last_inputs)

    # dL/dWhy and dL/dby
    d_Why = d_y @ self.last_hs[n].T
    d_by = d_y

    # Initialize dL/dWhh, dL/dWxh, and dL/dbh to zero
    d_Whh = np.zeros(self.Whh.shape)
    d_Wxh = np.zeros(self.Wxh.shape)
    d_bh = np.zeros(self.bh.shape)

    # Calculate dL/dh for the last h
    d_h = self.Why.T @ d_y

    # Backpropagate through time
    for t in reversed(range(n)):
      temp = ((1 - self.last_hs[t + 1] ** 2) * d_h)

      d_bh += temp
      d_Whh += temp @ self.last_hs[t].T
      d_Wxh += temp @ self.last_inputs[t].T
      d_h = self.Whh @ temp

    # Prevent exploding gradients
    for d in [d_Wxh, d_Whh, d_Why, d_bh, d_by]:
      np.clip(d, -1, 1, out=d)

    # Update weights and biases
    self.Whh -= learn_rate * d_Whh
    self.Wxh -= learn_rate * d_Wxh
    self.Why -= learn_rate * d_Why
    self.bh -= learn_rate * d_bh
    self.by -= learn_rate * d_by

In [ ]:
def softmax(xs):
  # Applies the Softmax Function to the input array
  return np.exp(xs) / sum(np.exp(xs))

In [ ]:
# Initialize RNN
rnn = RNN(vocab_size, 2)

inputs = createInputs("i am very good")
out, h = rnn.forward(inputs)
probabilities = softmax(out)
print(probabilities)

[[0.50000054]
 [0.49999946]]


In [ ]:
# Get loss and accuracy for the given data
# - data is a dictionary mapping text to True or False.
# - backprop determines if the backward phase should be run.
def processData(data, backprop=True):
  items = list(data.items())
  random.shuffle(items)

  loss = 0
  num_correct = 0

  for x, y in items:
    inputs = createInputs(x)
    target = int(y)

    # Forward
    out, _ = rnn.forward(inputs)
    probs = softmax(out)

    # Calculate loss / accuracy
    loss -= np.log(probs[target])
    num_correct += int(np.argmax(probs) == target)

    if backprop:
      # Build dL/dy
      d_L_d_y = probs
      d_L_d_y[target] -= 1

      # Backward
      rnn.backprop(d_L_d_y)

  return loss / len(data), num_correct / len(data)

In [ ]:
# Training cycle
for epoch in range(1000):
  train_loss, train_acc = processData(train_data)

  if epoch % 100 == 99:
    print('--- Epoch %d' % (epoch + 1))
    print('Train:\tLoss %.3f | Accuracy: %.3f' % (train_loss, train_acc))

    test_loss, test_acc = processData(test_data, backprop=False)
    print('Test:\tLoss %.3f | Accuracy: %.3f' % (test_loss, test_acc))

--- Epoch 100
Train:	Loss 0.688 | Accuracy: 0.552
Test:	Loss 0.699 | Accuracy: 0.500
--- Epoch 200
Train:	Loss 0.647 | Accuracy: 0.672
Test:	Loss 0.748 | Accuracy: 0.650
--- Epoch 300
Train:	Loss 0.178 | Accuracy: 0.948
Test:	Loss 0.096 | Accuracy: 1.000
--- Epoch 400
Train:	Loss 0.020 | Accuracy: 1.000
Test:	Loss 0.015 | Accuracy: 1.000
--- Epoch 500
Train:	Loss 0.004 | Accuracy: 1.000
Test:	Loss 0.005 | Accuracy: 1.000
--- Epoch 600
Train:	Loss 0.002 | Accuracy: 1.000
Test:	Loss 0.004 | Accuracy: 1.000
--- Epoch 700
Train:	Loss 0.002 | Accuracy: 1.000
Test:	Loss 0.003 | Accuracy: 1.000
--- Epoch 800
Train:	Loss 0.001 | Accuracy: 1.000
Test:	Loss 0.002 | Accuracy: 1.000
--- Epoch 900
Train:	Loss 0.001 | Accuracy: 1.000
Test:	Loss 0.002 | Accuracy: 1.000
--- Epoch 1000
Train:	Loss 0.001 | Accuracy: 1.000
Test:	Loss 0.001 | Accuracy: 1.000


In [ ]:
# Check available words
vocab

['is',
 'happy',
 'was',
 'very',
 'at',
 'right',
 'all',
 'earlier',
 'am',
 'not',
 'good',
 'now',
 'sad',
 'and',
 'this',
 'or',
 'i',
 'bad']

In [ ]:
my_new_data = {
    'this is good and not sad': True,
    'i am not bad and not good': False,
    'this is very sad and not bad': False,
    'i am very good and very happy': True,
    'this is not very sad and very good': True,
    'i am not at all good or happy': False,
    'this was very happy and not sad earlier': True,
    'i was not bad and not happy earlier': False,
    'this is not very good and not happy': False,
    'i am not happy and not sad': False,
}

In [ ]:
my_new_data_loss, my_new_data_acc = processData(my_new_data, backprop=False)
print('Test:\tLoss %.3f | Accuracy: %.3f' % (my_new_data_loss, my_new_data_acc))

Test:	Loss 0.624 | Accuracy: 0.900
